In [ ]:
from pathlib import Path
from os import environ
db_path = Path.cwd().parent.parent / "male_t20.db"
environ["DATABASE_URL"] = f"sqlite:///{db_path}"

In [ ]:
%load_ext sql
#%sql sqlite://///home/mike/extracover/male_t20.db
%sql

In [ ]:
%config SqlMagic.displaylimit=20

## Run distribution by phase of innings

In [ ]:
qry = """
WITH phased AS (
    SELECT
        *
    ,   CASE
        WHEN over BETWEEN 0 AND 5 THEN '0 powerplay'
        WHEN over BETWEEN 6 AND 17 THEN '1 mid-innings'
        ELSE '2 last two'
        END as phase
    FROM balls
)
SELECT phase, batter_runs, COUNT(*) AS count
FROM phased
WHERE innings < 2
AND extra_type = ''
AND batter_runs in (0,1,2,3,4,6)
GROUP BY phase, batter_runs
"""
result = %sql {{qry}}
df=result.DataFrame()
df.pivot(columns="phase", index="batter_runs", values="count")

## Average runs by over, first innings

In [ ]:
sql= """
WITH over_runs AS (
SELECT
	over
,	CAST(SUM(batter_runs) + SUM(extra_runs) AS FLOAT) AS runs
FROM balls
WHERE innings = 0
GROUP BY match_id, over
HAVING MAX(ball) = 5
)
SELECT
	over
,	SUM(runs) / COUNT(*) AS avg_runs
FROM over_runs
GROUP BY over
ORDER BY over
"""

result=%sql {{sql}}
df=result.DataFrame()
df.plot(x="over", y="avg_runs", kind="bar")

## Batsman "Aggression"

First, what's the average *batter only* runs per ball in each over?

In [ ]:
sql= """
WITH over_runs AS (
SELECT
	over
,	CAST(SUM(batter_runs) AS FLOAT) AS runs
FROM balls
WHERE innings = 0
AND extra_type = ''
GROUP BY match_id, over
HAVING MAX(ball) = 5
)
SELECT
	over
,	SUM(runs) / COUNT(*) / 6 AS avg_runs
FROM over_runs
GROUP BY over
ORDER BY over
"""

result=%sql {{sql}}
df=result.DataFrame()
df.plot(x="over", y="avg_runs", kind="bar", figsize=(7,2))

Now, take a batsman - #137 CH Gayle has the most ball faced (and is, or was, notoriously aggressive)

In [ ]:
sql = """SELECT
	over
,	COUNT(*)
,	SUM(CAST(batter_runs AS FLOAT)) / COUNT(*) AS avg_runs
FROM balls
WHERE batter = 137
AND innings < 2
AND extra_type = ''
GROUP BY over
ORDER BY over"""

result=%sql {{sql}}
df_gayle=result.DataFrame()
gayle = df_gayle["avg_runs"] - df["avg_runs"]
gayle.plot(kind="bar", x="over", y="avg_runs", figsize=(7,2))
gayle.sum()

...or Virat Kohli (coincidentally #138 at time of computing)

In [ ]:
sql = """
SELECT
	over
,	COUNT(*)
,	SUM(CAST(batter_runs AS FLOAT)) / COUNT(*) AS avg_runs
FROM balls
WHERE batter = 138
GROUP BY over
ORDER BY over"""

result=%sql {{sql}}
df_kohli=result.DataFrame()
kohli = df_kohli["avg_runs"] - df["avg_runs"]
kohli.plot(kind="bar", x="over", y="avg_runs", figsize=(7,2))

Can we (usefully) fit some kind of curve (straight line) to that?

Or perhaps just smooth the by-over values somehow, perhaps by some weighted average of adjacent (one? more?) values?

Or even just apply the average of the differences to the averall runs-per-ball curve, bumping the probabilities for each score as necessary?

Also, how much do those figures vary over time, or match-by-match, I wonder? Are there discernible trends?

## Average First Innings Totals by City/Venue

In [ ]:
sql = """
with first_inns_tots as (
	select b.match_id, m.city, m.venue, sum(b.batter_runs + b.extra_runs) as inns_tot
	from balls b join matches m on m.rowid = b.match_id
	where b.innings = 0
	and match_id in (
		select match_id from balls where innings = 0 group by match_id having count(*) >= 120
	)
	group by match_id
)
select city, venue, count(*), avg(inns_tot), min(inns_tot), max(inns_tot)
from first_inns_tots group by city, venue having count(*) >= 20 order by count(*) desc
"""
%sql {{sql}}

## Scoring shot frequencies by batsman & over

for players having faced  	enough deliveries to get usual distributions (first attempt: 1000 balls faced)

In [ ]:
# NOTE: this looks like it would benefit from being transformed into a view, but I tried it and it was horribly slower: 26s vs 0.3s

sql = """
SELECT
	over
,	batter
,	(SUM(CASE WHEN batter_runs = 0 THEN 1 END) + 0.0) / SUM(Count(*)) OVER (PARTITION BY batter, over) AS "0"
,	(SUM(CASE WHEN batter_runs = 1 THEN 1 END) + 0.0) / SUM(Count(*)) OVER (PARTITION BY batter, over) AS "1"
,	(SUM(CASE WHEN batter_runs = 2 THEN 1 END) + 0.0) / SUM(Count(*)) OVER (PARTITION BY batter, over) AS "2"
,	(SUM(CASE WHEN batter_runs = 3 THEN 1 END) + 0.0) / SUM(Count(*)) OVER (PARTITION BY batter, over) AS "3"
,	(SUM(CASE WHEN batter_runs = 4 THEN 1 END) + 0.0) / SUM(Count(*)) OVER (PARTITION BY batter, over) AS "4"
,	(SUM(CASE WHEN batter_runs = 5 THEN 1 END) + 0.0) / SUM(Count(*)) OVER (PARTITION BY batter, over) AS "5"
,	(SUM(CASE WHEN batter_runs = 6 THEN 1 END) + 0.0) / SUM(Count(*)) OVER (PARTITION BY batter, over) AS "6"
FROM
	balls
WHERE innings < 2
AND extra_type = ''
AND batter in (137, 138, 297)
GROUP BY over, batter
ORDER BY batter, over
"""

result = %sql {{sql}}
df = result.DataFrame()

In [ ]:
df[df["batter"]=="137"].plot(x="over", y=["0", "1", "2", "4", "6"],kind="bar", figsize=(7,2), stacked = True, legend="reverse")

In [ ]:
df[df["batter"]=="138"].plot(x="over", y=["0", "1", "2", "4", "6"],kind="bar", figsize=(7,2), stacked=True, legend="reverse")

In [ ]:
df[df["batter"]=="297"].plot(x="over", y=["0", "1", "2", "4", "6"],kind="bar", figsize=(7,2), stacked=True, legend="reverse")

## Batman behaviour in first 10 balls

A sort of "cautiousness quotient" ?

In [ ]:
sql = """
WITH seqs AS (
	SELECT
		ROW_NUMBER() OVER (PARTITION BY batter, match_id, innings ORDER BY match_id, over, ball_seq) AS row_num
	,	batter
	,	CAST(batter_runs AS FLOAT) AS batter_runs
	,	wicket_fell
	FROM balls
    WHERE extra_type = ''
    AND innings < 2
	AND batter IN (137, 138, 297)
)
, cte AS (
	SELECT
		batter
	,	batter_runs
	,	CASE WHEN row_num <= 10 THEN '0: first 10' ELSE '1: thereafter' END AS period
	,	wicket_fell
	FROM seqs
)
SELECT
	batter
,	period
,	AVG(batter_runs) AS avg_runs
,	CAST(SUM(wicket_fell) AS FLOAT) / COUNT(*) AS wicket_prob
FROM cte
GROUP BY batter, period
ORDER BY 1, 2
"""

%sql {{sql}}

## Batsman aggression factors

Determined thus:

* Reference all non-extra balls not in super overs where striker was not out
* derive the average runs per ball for all identified balls
* same calculation per batsman; limit to batters with some minimum # of balls faced and a minimum # of balls faced in each over
* calculate each batter's delta over the universal average for each qualified over
* aggression is the average of the deltas for all qualifying overs

## Bowler economy factor

As above, mostly, but from the bowler's point-of-view